Для начала обновитесь до torch-1.11.0 torchvision-0.12.0

In [ ]:
pip install --upgrade torch torchvision

     |████████████████████████████████| 750.6 MB 11 kB/s 
     |████████████████████████████████| 21.0 MB 1.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.11.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.11.0 which is incompatible.


Импортируем все, что нужно

In [ ]:
from PIL import Image, ImageFile
import numpy as np
import pandas as pd
from tqdm import tqdm
import shutil
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
import torchvision.models as models
from torchvision import transforms

from sklearn.metrics import accuracy_score, f1_score

Создаем папочки, в которые будем делить картинки

In [6]:
class_names = os.listdir('/content/drive/MyDrive/train')
os.mkdir('/content/images')
os.mkdir('/content/images/train_images')
os.mkdir('/content/images/valid_images')
for i in range(len(class_names)):
  os.mkdir(f'/content/images/train_images/{class_names[i]}')
  os.mkdir(f'/content/images/valid_images/{class_names[i]}')

Делим картинки, каждую пятую в валидационный сет

In [7]:
path_1 = '/content/drive/MyDrive/train'
path_2 = '/content/images'
for class_ in class_names:
  for i, name_img in enumerate(os.listdir(f'/content/drive/MyDrive/train/{class_}')):
    if i % 5 == 0:
      shutil.copy(f'{path_1}/{class_}/{name_img}', f'{path_2}/valid_images/{class_}/{name_img}')
    else:
      shutil.copy(f'{path_1}/{class_}/{name_img}', f'{path_2}/train_images/{class_}/{name_img}')
      
os.rename("/content/images/train_images/drova", "/content/images/train_images/0")
os.rename("/content/images/valid_images/drova", "/content/images/valid_images/0")

делаем даталоадеры

In [16]:
img_transforms = transforms.Compose([
    transforms.Resize((64,64)), 
    transforms.AutoAugment(),   
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225]),
    ])

In [17]:
data_dir = '/content/images'
image_datasets = {x: torchvision.datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform=img_transforms)
                  for x in ['train_images', 'valid_images']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=2)
              for x in ['train_images', 'valid_images']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train_images', 'valid_images']}
class_names = image_datasets['train_images'].classes

Берем предобученную ConvNext и заменяем выходной слой на слой с тремя выходами

In [10]:
convnext = models.convnext_large(pretrained=True)

Downloading: "https://download.pytorch.org/models/convnext_large-ea097f82.pth" to /root/.cache/torch/hub/checkpoints/convnext_large-ea097f82.pth


  0%|          | 0.00/755M [00:00<?, ?B/s]

In [11]:
for name, param in convnext.named_parameters():
  param.requires_grad = False

In [12]:
convnext.classifier[2] = nn.Linear(in_features=1536, out_features=3, bias=True)

Объявляем оптимизатор, функцию потерь и пишем функцию обучения

In [13]:
optimizer = optim.NAdam(convnext.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

In [14]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20):
    best_metrics = 0
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for i, batch in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            inputs, targets = batch
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)

        model.eval()
        num_correct = 0 
        num_examples = 0
        metrics = 0
        extra_metric = 0
        for i, batch in enumerate(tqdm(val_loader)):
            inputs, targets = batch
            output = model(inputs)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
              
            y_pred = torch.max(F.softmax(output, dim=1), dim=1)[1].numpy()
            metrics += accuracy_score(targets.numpy(), y_pred)
            extra_metric += f1_score(targets.numpy(), y_pred, average='macro')
        valid_loss /= len(val_loader.dataset)
        metrics /= len(val_loader)
        extra_metric /= len(val_loader)

        if metrics > best_metrics:
          print('New best model with test accuracy:', metrics)
          torch.save(model, "/content/convnext_weight")
          best_metrics = metrics

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}, f1_score = {:.2f}'.format(epoch, training_loss,
        valid_loss, metrics, extra_metric))


Учим

In [23]:
train(convnext, optimizer, loss_fn, dataloaders['train_images'], dataloaders['valid_images'], epochs=10)

100%|██████████| 29/29 [00:28<00:00,  1.02it/s]


New best model with test accuracy: 0.9310344827586207
Epoch: 0, Training Loss: 0.19, Validation Loss: 0.18, accuracy = 0.93, f1_score = 0.91


100%|██████████| 29/29 [00:29<00:00,  1.02s/it]


New best model with test accuracy: 0.9396551724137931
Epoch: 1, Training Loss: 0.21, Validation Loss: 0.19, accuracy = 0.94, f1_score = 0.90


100%|██████████| 29/29 [00:27<00:00,  1.07it/s]


...

Epoch: 7, Training Loss: 0.20, Validation Loss: 0.27, accuracy = 0.91, f1_score = 0.85


100%|██████████| 29/29 [00:28<00:00,  1.01it/s]


Epoch: 8, Training Loss: 0.17, Validation Loss: 0.20, accuracy = 0.91, f1_score = 0.86


100%|██████████| 29/29 [00:26<00:00,  1.08it/s]

Epoch: 9, Training Loss: 0.18, Validation Loss: 0.17, accuracy = 0.94, f1_score = 0.92


Загружаем лучшие веса

In [ ]:
convnext_custom = torch.load('/content/convnext_weight')

Объявляем трансформ без автоаугментации и используем его в предсказании

In [ ]:
test_transforms = transforms.Compose([
    transforms.Resize((64,64)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225]),
    ])

In [ ]:
path_test = '/content/drive/MyDrive/wood/test'
id = []
predict_class = []
convnext_custom.eval()
for img in os.listdir(path_test):
  id.append(int(img.split('.')[0]))

  img_test = Image.open(f'{path_test}/{img}')
  img_test = test_transforms(img_test)
  img_test = torch.unsqueeze(img_test, 0)
  
  prediction = F.softmax(convnext_custom(img_test))
  prediction = prediction.argmax()
  predict_class.append(class_names[prediction])

сохраняем для сабмита

In [ ]:
pd.DataFrame({'id': id, 'class': predict_class}).sort_values(by='id').to_csv('autoaugmented60.csv', index=False)

In [ ]:
!cp /content/convnext_weight /content/drive/MyDrive/woodhack/convnext_weight

In [ ]:
!cp /content/autoaugmented60.csv /content/drive/MyDrive/woodhack/convnext_weight/autoaugmented.csv